In [1]:
num_qubits_list = [7] # [7] # [5] # [9]
num_trial = 20 # 3 # 1

In [2]:
from qiskit_aer import AerSimulator
from qiskit import Aer
from qiskit import transpile

In [3]:
def change_bin(bit_list) :
    value = 0
    for element in bit_list[::-1]:
        value = value*2
        value += element
    return value


def generate_binaries(n):

    L=[[] for j in range(n)]

    for i in range(pow(2,n-1)) :
        ca = list(bin(i))[2:]
        for j in range(len(ca)) :
            ca[j] = int(ca[j])
        if len(ca) < n-1 :
            ca = [0 for k in range(n - 1 - len(ca))] + ca

        num_one = 0
        for i in range(0,n-1):
            if ca[i] == 1:
                num_one += 1

        #print(num_one)
        num_anc = n-1
        if num_one < n/2:
            for i in range(0,n-1):
                if ca[i] == 1:
                    L[i+1].append(change_bin(ca))
        else: 
            L[0].append(change_bin(ca))
            for i in range(0,n-1):
                if ca[i] == 0:
                    L[i+1].append(change_bin(ca)) 
    return L

In [4]:
from qiskit_aer.noise import NoiseModel, pauli_error

In [5]:
p_err = 0.5 # 0.4 # 0.2 # 0.1 # 0.03 # 0.25 # 0.1 # 0. 

error_inv_x = pauli_error([('X', 1 - p_err), ('I', p_err)])
error_inv_z = pauli_error([('Z', 1 - p_err), ('I', p_err)])

In [6]:
import qiskit.quantum_info as qi

x_op = qi.Operator([[0,1],[1,0]])
z_op = qi.Operator([[1,0],[0,-1]])

noise_model = NoiseModel()

# noise_model.add_all_qubit_quantum_error(error_x, 'bit-flip')
noise_model.add_all_qubit_quantum_error(error_inv_x, 'bit-flip')
noise_model.add_all_qubit_quantum_error(error_inv_z, 'phase-flip')

noise_model.add_basis_gates(['unitary'])

# noise_model.add_quantum_error(error_x, 'x', [1])

In [7]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi
import numpy as np
from qiskit_aer.noise import pauli_error
from tqdm import tqdm

def string_1s(n):
    ssss = '1'
    for i in range(n-1):
        ssss+='1'
    return ssss

def string_0s(n):
    ssss = '0'
    for i in range(n-1):
        ssss+='0'
    return ssss

def return_to_bits(bit_string) :
    b = int(bit_string,2)
    return b

error_type = 'phase-flip' # 'bit-flip'
shots_num = 10000

counts_list_list = []
for n in num_qubits_list:
    
    ##### number of qubits #####

    ##### number of shots #####

    ##### circuit setting ######
    q0 = QuantumRegister(n, 'qm') # message
    q1 = QuantumRegister(n-1, 'qa') # ancillar
    c0 = ClassicalRegister(n, 'cm')
    c1 = ClassicalRegister(n-1, 'ca')
    circuit = QuantumCircuit(q0, q1, c0, c1)

    ##### encoding a physical qubit into a logical qubit #####
    for i in range(1,n):
        circuit.cx(q0[0],q0[i])
        
    ##### convert phase flip into bit flip #####
    if error_type == 'phase-flip':
        circuit.h(q0)
    circuit.barrier(q0, q1)
    
    ##### introduce errors #####
    if error_type == 'phase-flip':
        circuit.unitary(z_op, q0, label='phase-flip')
#         circuit.z(q0[0])
#         circuit.z(q0[1])
    elif error_type == 'bit-flip':
        circuit.unitary(x_op, q0, label='bit-flip')
    circuit.barrier(q0, q1)
    
    ##### convert phase flip into bit flip #####
    if error_type == 'phase-flip':
        circuit.h(q0)
    
    ##### make ancilla qubits #####
    for i in range(0, n-1):
        circuit.cx(q0[0], q1[i])
        circuit.cx(q0[i+1], q1[i])
    circuit.barrier(q0, q1)
    

    ##### measure ancillas #####
    circuit.measure(q1,c1)
    circuit.barrier(q0,q1)

    ##### correction circuit #####
    num_list_list = generate_binaries(n)
    for j_qubit, num_list in enumerate(num_list_list):
        for num in num_list:
            circuit.x(q0[j_qubit]).c_if(c1,num)
        
    circuit.barrier(q0,q1)
    
    ##### result measurement #####
    circuit.measure(q0,c0)

    ##################################################
#     backend = Aer.get_backend('qasm_simulator', noise_model=noise_model)

#     from qiskit.providers.fake_provider  import FakeMumbai
#     backend = FakeMumbai()

#     backend = AerSimulator()
    backend = AerSimulator(noise_model=noise_model)
    tcirc = transpile(circuit, backend)

    ##################################################

    n_data = []
    counts_list = []
    for iteration_number in tqdm(range(num_trial)):
        
        iteration_data = []
        
        string_0 = string_0s(n)
        string_1 = string_1s(n)
        
        job = backend.run(tcirc, shots=shots_num)
        result = job.result()
        counts = result.get_counts(tcirc)
        
        counts_list.append(counts)
        print(f"n={n}: {counts}")
    
    counts_list_list.append(counts_list)

#         bit_total_result = list(counts.keys())
#         num_000 = 0
#         for bits in bit_total_result :
#             if bits.split()[1] == string_0 :
#                 num_000 += counts[bits]
#         print("Measured number of outcome '" + string_0 + "':", num_000)

#         num_111 = 0
#         for bits in bit_total_result :
#             if bits.split()[1] == string_1 :
#                 num_111 += counts[bits]
#         print("Measured number of outcome '" + string_1 + "':", num_111)
            
#         ########    
        
#         num_eff000 = 0
#         for bits in bit_total_result :
#             if bin(return_to_bits(bits.split()[1]) ^ return_to_bits(string_0))[2:].count('1') < (n+1)/2 :
#             #if bits.split()[1] == '000' or bits.split()[1] == '001' or bits.split()[1] == '010' or bits.split()[1] == '100' :
#                 num_eff000 += counts[bits]
#         print("Measured number of outcome considered as '" + string_0 + "':", num_eff000)

#         num_eff111 = 0
#         for bits in bit_total_result :
#             if bin(return_to_bits(bits.split()[1]) ^ return_to_bits(string_1))[2:].count('1') < (n+1)/2 :
#             #if bits.split()[1] == '111' or bits.split()[1] == '110' or bits.split()[1] == '101' or bits.split()[1] == '011' :
#                 num_eff111 += counts[bits]
#         print("Measured number of outcome considered as '" + string_1 + "':", num_eff111)
        
#         ########
        
#         iteration_data.append([num_000, num_111, num_eff000, num_eff111])
#         n_data.append(iteration_data)
        
#     np.save(str(n) + "_qubits.npy", np.array(n_data))

 10%|█████████▍                                                                                    | 2/20 [00:10<01:32,  5.14s/it]

n=5: {'0100 11111': 321, '1010 00000': 309, '0000 00000': 324, '0110 11111': 308, '1110 00000': 301, '0010 11111': 307, '1110 11111': 305, '1101 11111': 341, '0101 00000': 337, '1011 11111': 307, '0100 00000': 318, '0001 11111': 320, '1001 11111': 326, '1100 11111': 303, '0011 00000': 318, '1101 00000': 323, '0001 00000': 289, '1111 00000': 287, '1011 00000': 307, '1001 00000': 327, '0000 11111': 328, '0010 00000': 328, '0011 11111': 279, '0111 00000': 286, '0110 00000': 314, '0101 11111': 323, '1000 00000': 280, '1100 00000': 315, '0111 11111': 320, '1000 11111': 322, '1010 11111': 304, '1111 11111': 323}
n=5: {'1001 00000': 297, '0000 11111': 331, '0110 00000': 337, '0010 11111': 311, '1011 00000': 320, '1101 11111': 311, '0101 00000': 305, '1100 00000': 313, '0010 00000': 335, '1110 00000': 318, '1000 00000': 296, '1010 11111': 309, '1101 00000': 331, '0011 11111': 293, '0111 00000': 309, '0100 00000': 306, '1011 11111': 281, '1100 11111': 288, '0011 00000': 308, '0101 11111': 320, 

 15%|██████████████                                                                                | 3/20 [00:15<01:30,  5.30s/it]

n=5: {'0110 11111': 306, '0000 00000': 294, '0001 00000': 325, '1110 11111': 297, '0001 11111': 328, '1101 00000': 298, '0111 11111': 307, '0111 00000': 304, '0011 11111': 322, '1111 00000': 314, '1010 11111': 274, '0101 11111': 323, '0101 00000': 314, '1101 11111': 327, '1010 00000': 333, '1100 00000': 317, '1011 00000': 303, '0000 11111': 296, '1001 00000': 304, '0100 11111': 359, '1001 11111': 327, '1110 00000': 312, '1111 11111': 314, '1100 11111': 316, '0011 00000': 279, '1000 11111': 330, '0110 00000': 303, '0010 11111': 305, '0010 00000': 333, '1000 00000': 324, '1011 11111': 309, '0100 00000': 303}


 25%|███████████████████████▌                                                                      | 5/20 [00:35<02:02,  8.17s/it]

...

n=5: {'1100 00000': 311, '0001 11111': 348, '0010 11111': 327, '1011 00000': 303, '0100 11111': 299, '0110 00000': 303, '0011 00000': 313, '1100 11111': 332, '0000 00000': 325, '0110 11111': 303, '1010 00000': 322, '0101 00000': 300, '1101 11111': 310, '1010 11111': 304, '1110 11111': 323, '1101 00000': 306, '1110 00000': 311, '0111 11111': 289, '0111 00000': 304, '0011 11111': 331, '0101 11111': 317, '1000 00000': 293, '0010 00000': 309, '0001 00000': 330, '1111 11111': 285, '0100 00000': 305, '1011 11111': 325, '0000 11111': 298, '1001 00000': 308, '1000 11111': 326, '1111 00000': 330, '1001 11111': 310}


 90%|███████████████████████████████████████████████████████████████████████████████████▋         | 18/20 [02:33<00:18,  9.11s/it]

n=5: {'0001 00000': 309, '0010 11111': 334, '0111 11111': 336, '0010 00000': 296, '0101 11111': 316, '0101 00000': 329, '1101 11111': 312, '1010 00000': 325, '0100 11111': 315, '0001 11111': 311, '1111 11111': 311, '1001 11111': 334, '0011 11111': 334, '0111 00000': 287, '0000 00000': 274, '0110 11111': 302, '1011 11111': 317, '0100 00000': 323, '1110 00000': 331, '1001 00000': 322, '0000 11111': 290, '1100 11111': 337, '0011 00000': 315, '1111 00000': 292, '1100 00000': 291, '1101 00000': 315, '1000 11111': 276, '1000 00000': 322, '1110 11111': 320, '0110 00000': 327, '1010 11111': 292, '1011 00000': 305}
n=5: {'1101 00000': 307, '1000 11111': 296, '1100 00000': 303, '1101 11111': 309, '0101 00000': 310, '0100 11111': 330, '0010 11111': 317, '1011 00000': 307, '0000 00000': 304, '0110 11111': 340, '1111 00000': 315, '0101 11111': 319, '0011 00000': 318, '1100 11111': 321, '1001 11111': 323, '0100 00000': 313, '1011 11111': 278, '1010 00000': 296, '0001 00000': 324, '0010 00000': 308, 

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [02:51<00:00,  8.60s/it]

n=5: {'1000 11111': 288, '1001 00000': 297, '0000 11111': 310, '1100 00000': 311, '0001 11111': 305, '0101 00000': 318, '1101 11111': 299, '1010 11111': 323, '1001 11111': 304, '1100 11111': 337, '0011 00000': 344, '1111 00000': 292, '1011 11111': 294, '0100 00000': 323, '0100 11111': 320, '1000 00000': 312, '1011 00000': 278, '0010 11111': 328, '1101 00000': 335, '1010 00000': 342, '1111 11111': 285, '0011 11111': 314, '0111 00000': 323, '0110 11111': 307, '0000 00000': 298, '0010 00000': 307, '0110 00000': 317, '1110 00000': 310, '0001 00000': 312, '0101 11111': 313, '0111 11111': 322, '1110 11111': 332}


In [14]:
# circuit.draw('mpl', filename='circuit-test-019.png')

In [15]:
# counts = counts_list_list[0][0]

In [16]:
def counts_to_merged_counts(counts):

    data_list = set([k.split()[-1] for k in counts.keys()])
    word_lengths = [len(word) for word in data_list]
    nq = word_lengths[0]
#     nq = num_qubits_list[0]
    assert all([nq == word for word in word_lengths])

    all_0 = '0' * nq
    all_1 = '1' * nq
    counts_data = {all_0:0, all_1:0}
    for k, v in counts.items():
        if k[-nq:] == all_0:
            counts_data[all_0] += v
        else:
            counts_data[all_1] += v
            
#     return counts_data
    return counts_data[all_0], counts_data[all_1]

In [17]:
# counts_to_logical_err

# counts_data

In [18]:
counts_trials = np.empty((num_trial,2), dtype=np.int64)
for j_trial in range(num_trial):
    merged_counts = counts_to_merged_counts(counts_list_list[0][j_trial])
    counts_trials[j_trial] = merged_counts
    
counts_trials

array([[4963, 5037],
       [5038, 4962],
       [4960, 5040],
       [5035, 4965],
       [4996, 5004],
       [4965, 5035],
       [5112, 4888],
       [4945, 5055],
       [4979, 5021],
       [5041, 4959],
       [5052, 4948],
       [4938, 5062],
       [5070, 4930],
       [4922, 5078],
       [5094, 4906],
       [5031, 4969],
       [4973, 5027],
       [4963, 5037],
       [4951, 5049],
       [5019, 4981]], dtype=int64)

In [19]:
# np.save(f'counts-000-111_{error_type}_p-{p_err:.3g}_numqubits-{num_qubits_list[0]}_test-015.npy', counts_trials)

In [17]:
# assert sum(counts_data.values()) == shots_num

In [ ]:
p_logical_err = 1 - counts_data[all_0] / sum(counts_data.values())

In [ ]:
nq_div_2, r_ = divmod(nq, 2)
assert r_ == 1

In [ ]:
nq_div_2

In [17]:
from math import comb

In [18]:
comb(3,2)

3

In [19]:
p_logical_err_theor = 0.
for jj in range(nq_div_2+1):
    p_logical_err_theor += comb(nq,jj) * pow(p_err,nq-jj) * pow(1-p_err,jj)

In [20]:
p_logical_err, p_logical_err_theor

(0.027900000000000036, 0.028000000000000008)

In [21]:
from qiskit.visualization import plot_histogram

In [24]:
# plot_histogram(counts_list_list[0][0], filename=f'hist-7-rep-p-{p_err:.3g}-test-002.png')

In [23]:
tcirc.draw()

░ ┌──────────┐ ░                      ░        ░ ┌───────┐»
qm_0: ──■────■───░─┤ bit-flip ├─░───■────■─────────────░────────░─┤ Rz(π) ├»
      ┌─┴─┐  │   ░ ├──────────┤ ░   │    │             ░        ░ └───╥───┘»
qm_1: ┤ X ├──┼───░─┤ bit-flip ├─░───┼────┼────■────────░────────░─────╫────»
      └───┘┌─┴─┐ ░ ├──────────┤ ░   │    │    │        ░        ░     ║    »
qm_2: ─────┤ X ├─░─┤ bit-flip ├─░───┼────┼────┼────■───░────────░─────╫────»
           └───┘ ░ └──────────┘ ░ ┌─┴─┐  │  ┌─┴─┐  │   ░ ┌─┐    ░     ║    »
qa_0: ───────────░──────────────░─┤ X ├──┼──┤ X ├──┼───░─┤M├────░─────╫────»
                 ░              ░ └───┘┌─┴─┐└───┘┌─┴─┐ ░ └╥┘┌─┐ ░     ║    »
qa_1: ───────────░──────────────░──────┤ X ├─────┤ X ├─░──╫─┤M├─░─────╫────»
                 ░              ░      └───┘     └───┘ ░  ║ └╥┘ ░     ║    »
cm: 3/════════════════════════════════════════════════════╬══╬════════╬════»
                                                          ║  ║     ┌──╨──┐ »
ca: 2/════════════════════════════════════════════════════╩══╩═════╡ 0x3 ╞═»
                                                          0  1     └─────┘ »
«       ┌────┐┌────────┐ ┌────┐┌────────┐                                 »
«qm_0: ─┤ √X ├┤ Rz(2π) ├─┤ √X ├┤ Rz(3π) ├─────────────────────────────────»
«       └─╥──┘└───╥────┘ └─╥──┘└───╥────┘┌───────┐ ┌────┐┌────────┐ ┌────┐»
«qm_1: ───╫───────╫────────╫───────╫─────┤ Rz(π) ├─┤ √X ├┤ Rz(2π) ├─┤ √X ├»
«         ║       ║        ║       ║     └───╥───┘ └─╥──┘└───╥────┘ └─╥──┘»
«qm_2: ───╫───────╫────────╫───────╫─────────╫───────╫───────╫────────╫───»
«         ║       ║        ║       ║         ║       ║       ║        ║   »
«qa_0: ───╫───────╫────────╫───────╫─────────╫───────╫───────╫────────╫───»
«         ║       ║        ║       ║         ║       ║       ║        ║   »
«qa_1: ───╫───────╫────────╫───────╫─────────╫───────╫───────╫────────╫───»
«         ║       ║        ║       ║         ║       ║       ║        ║   »
«cm: 3/═══╬═══════╬════════╬═══════╬═════════╬═══════╬═══════╬════════╬═══»
«      ┌──╨──┐ ┌──╨──┐  ┌──╨──┐ ┌──╨──┐   ┌──╨──┐ ┌──╨──┐ ┌──╨──┐  ┌──╨──┐»
«ca: 2/╡ 0x3 ╞═╡ 0x3 ╞══╡ 0x3 ╞═╡ 0x3 ╞═══╡ 0x1 ╞═╡ 0x1 ╞═╡ 0x1 ╞══╡ 0x1 ╞»
«      └─────┘ └─────┘  └─────┘ └─────┘   └─────┘ └─────┘ └─────┘  └─────┘»
«                                                            ░ ┌─┐      
«qm_0: ──────────────────────────────────────────────────────░─┤M├──────
«      ┌────────┐                                            ░ └╥┘┌─┐   
«qm_1: ┤ Rz(3π) ├────────────────────────────────────────────░──╫─┤M├───
«      └───╥────┘┌───────┐ ┌────┐┌────────┐ ┌────┐┌────────┐ ░  ║ └╥┘┌─┐
«qm_2: ────╫─────┤ Rz(π) ├─┤ √X ├┤ Rz(2π) ├─┤ √X ├┤ Rz(3π) ├─░──╫──╫─┤M├
«          ║     └───╥───┘ └─╥──┘└───╥────┘ └─╥──┘└───╥────┘ ░  ║  ║ └╥┘
«qa_0: ────╫─────────╫───────╫───────╫────────╫───────╫──────░──╫──╫──╫─
«          ║         ║       ║       ║        ║       ║      ░  ║  ║  ║ 
«qa_1: ────╫─────────╫───────╫───────╫────────╫───────╫──────░──╫──╫──╫─
«          ║         ║       ║       ║        ║       ║      ░  ║  ║  ║ 
«cm: 3/════╬═════════╬═══════╬═══════╬════════╬═══════╬═════════╩══╩══╩═
«       ┌──╨──┐   ┌──╨──┐ ┌──╨──┐ ┌──╨──┐  ┌──╨──┐ ┌──╨──┐      0  1  2 
«ca: 2/═╡ 0x1 ╞═══╡ 0x2 ╞═╡ 0x2 ╞═╡ 0x2 ╞══╡ 0x2 ╞═╡ 0x2 ╞══════════════
«       └─────┘   └─────┘ └─────┘ └─────┘  └─────┘ └─────┘